In [58]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.feature_extraction.text import CountVectorizer
import unidecode
import re
from sklearn.decomposition import TruncatedSVD
# import matplotlib.pyplot as plt
# import matplotlib.patches as mpatches
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.stem.snowball import SnowballStemmer
import datetime
import time

In [59]:
data = pd.read_csv("../Data procesada/data_audio_concatenada.csv")
data.drop(columns=['Unnamed: 0'],inplace=True)
data

,folder,path_audio,file,date,text_complete
0,C5N,../Data/Audio/C5N/C5N2020-08-11-17-59-23.wav,C5N2020-08-11-17-59-23.wav,2020-08-11 17:59:23,el doctor adolfo ruiz stein que había dicho qu...
1,C5N,../Data/Audio/C5N/C5N2020-08-14-17-55-49.wav,C5N2020-08-14-17-55-49.wav,2020-08-14 17:55:49,a hola hola muy buenas tardes bienvenidos al d...
2,TN,../Data/Audio/TN/TN2020-08-11-19-01-34.wav,TN2020-08-11-19-01-34.wav,2020-08-11 19:01:34,de ciento cincuenta mil casos corresponder al ...
3,TN,../Data/Audio/TN/TN2020-08-14-18-59-25.wav,TN2020-08-14-18-59-25.wav,2020-08-14 18:59:25,de los colores que se habían ido arrojó de las...


In [60]:
aux = pd.Series(data['file'].apply(lambda x: x.replace(".wav","").split("-")[-6:]))
aux = aux.apply(lambda x: [int(a) if len(a)<=4 else int(a[-4:]) for a in x])
data['date'] = aux.apply(lambda x: datetime.datetime(x[0], x[1], x[2], hour=x[3], minute=x[4], second=x[5]))
file = data['folder'] +"-"+ data['date'].apply(lambda x: str(x.month)+"-"+str(x.day))
data['day'] = data['date'].apply(lambda x: str(x.day))

In [61]:
# data=data.loc[data['day']=="14",:]
# data.reset_index(drop=True,inplace=True)

In [62]:
textos = data.loc[:,'text_complete']

textos_limpios=[];
for t in textos:
    t_lower_no_accents=unidecode.unidecode(t.lower()); # sacamos acentos y llevamos a minuscula
    t_lower_no_accents_no_punkt=re.sub(r'([^\s\w]|_)+','',t_lower_no_accents); # quitamos signos de puntuacion usando una regex que reemplaza todo lo q no sean espacios o palabras por un string vacio
    textos_limpios.append(t_lower_no_accents_no_punkt)

In [63]:
# Defimimos una función que aplica stemming a una frase en castellano:
def stemfraseesp(frase):    
    token_words=word_tokenize(frase)
    token_words
    stem_sentence=[]    
    spanishStemmer=SnowballStemmer("spanish",ignore_stopwords=True)
    for word in token_words:
        stem_sentence.append(spanishStemmer.stem(word))
        stem_sentence.append(" ")
    return "".join(stem_sentence)

----------

In [64]:
stopwords_sp = stopwords.words('spanish');

In [65]:
texto_completo = "".join(data['text_complete'])
texto_completo=unidecode.unidecode(texto_completo.lower()); # sacamos acentos y llevamos a minuscula
texto_completo=re.sub(r'([^\s\w]|_)+','',texto_completo); # quitamos signos de puntuacion usando una regex que reemplaza todo lo q no sean espacios o palabras por un string vacio

In [66]:
token_words=word_tokenize(texto_completo)
token_words = [i for i in token_words if i not in stopwords_sp]
stem_sentence=[]
spanishStemmer=SnowballStemmer("spanish",ignore_stopwords=True)
for word in token_words:
    stem_sentence.append(spanishStemmer.stem(word))
    stem_sentence.append(" ")

In [67]:
stem_sentence = pd.Series(stem_sentence)
mask = stem_sentence != " "
stem_sentence= stem_sentence.loc[mask]

In [68]:
stem_match = pd.DataFrame([token_words,stem_sentence]).T
stem_match.columns = ['original','stem']
stem_match = stem_match.groupby('stem')['original'].value_counts()
stem_match = pd.DataFrame(stem_match)
stem_match.columns = ['Count']
stem_match.reset_index(inplace=True)

In [69]:
stem_match = stem_match.sort_values(by='Count', ascending=False)
stem_match.drop_duplicates(subset=['stem'],keep='first', inplace=True)
stem_match.reset_index(drop=True,inplace=True)
print(stem_match.shape)
stem_match.head(20)

(4476, 3)


,stem,original,Count
0,mas,mas,310
1,si,si,288
2,tambi,tambien,245
3,ahor,ahora,167
4,buen,bueno,166
5,cuarenten,cuarentena,137
6,estan,estan,129
7,gobiern,gobierno,119
8,argentin,argentina,117
9,hoy,hoy,113


---------------

In [70]:
backup=textos_limpios
auxiliar = []
for t in textos_limpios:
    auxiliar.append(stemfraseesp(t))
    
textos_limpios=auxiliar

In [71]:
vectorizer=CountVectorizer(stop_words=stopwords_sp,lowercase=True,ngram_range=(1,1));

vectorizer.fit(textos_limpios);

feature_names = pd.DataFrame()

if len(vectorizer.get_feature_names()[0].split(' '))==2:
    aux = pd.Series(vectorizer.get_feature_names())
    aux = aux.str.split(' ',expand=True)
    aux.columns= ['word1','word2']
    aux = aux.merge(stem_match,how='left', left_on='word1',right_on='stem')
    aux = aux.merge(stem_match,how='left', left_on='word2',right_on='stem',suffixes=('_1','_2'))
    features_names= aux['original_1']+" "+aux['original_2']

else:
    aux = pd.DataFrame(vectorizer.get_feature_names())
    aux.columns=['word1']
    features_names= aux.merge(stem_match, how='left',left_on='word1',right_on='stem')['original']
    

CV_encoding=vectorizer.transform(textos_limpios);
pd.DataFrame(CV_encoding.todense(),columns=features_names)

original,aa,aaa,aahora,abajo,abasto,abdomen,abdominales,abertis,abierto,abismo,...,ymuy,you,youtube,ypor,yreino,yumbo,zaragoza,zelanda,zona,zonda
0,1,0,0,4,0,0,0,0,3,2,...,1,0,0,0,1,0,1,0,0,1
1,1,0,0,0,0,0,0,1,5,0,...,0,1,0,1,0,0,0,0,10,0
2,1,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,5,0
3,2,0,1,1,2,1,2,0,2,0,...,0,0,1,0,0,3,0,1,4,0


In [72]:
Tfidf_encoding=TfidfTransformer().fit_transform(CV_encoding);
pd.DataFrame(Tfidf_encoding.todense(),columns=features_names)

original,aa,aaa,aahora,abajo,abasto,abdomen,abdominales,abertis,abierto,abismo,...,ymuy,you,youtube,ypor,yreino,yumbo,zaragoza,zelanda,zona,zonda
0,0.002895,0.000000,0.000000,0.017495,0.000000,0.000000,0.000000,0.000000,0.008685,0.011095,...,0.005548,0.000000,0.000000,0.000000,0.005548,0.000000,0.005548,0.000000,0.000000,0.005548
1,0.002465,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.004723,0.012323,0.000000,...,0.000000,0.004723,0.000000,0.004723,0.000000,0.000000,0.000000,0.000000,0.030146,0.000000
2,0.005787,0.011089,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.005787,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.035390,0.000000
3,0.004920,0.000000,0.004714,0.003716,0.009427,0.004714,0.009427,0.000000,0.004920,0.000000,...,0.000000,0.000000,0.004714,0.000000,0.000000,0.014141,0.000000,0.004714,0.012035,0.000000


In [73]:
svd=TruncatedSVD(n_components=10);
P=svd.fit_transform(Tfidf_encoding)

# color = ['b','g','r','c']
# plt.figure()
# patches = []

# P = pd.DataFrame(P)
# P.iloc[:,0:2]
# i=0
# for texto in file.unique():
#     mask= file==texto
#     plt.plot(P.loc[mask,0], P.loc[mask,1], color[i]+"o", label=''.join(file[mask]))
#     i=i+1

# plt.legend()
# plt.xlabel('dimension 1')
# plt.ylabel('dimension 2')
# plt.show()

In [74]:
features_names

0             aa
1            aaa
2         aahora
3          abajo
4         abasto
          ...   
4456       yumbo
4457    zaragoza
4458     zelanda
4459        zona
4460       zonda
Name: original, Length: 4461, dtype: object

In [75]:
componentes_SVD=pd.DataFrame(features_names)
componentes_SVD.columns=['feature']
for i in range(0,len(svd.components_)):
    componentes_SVD = componentes_SVD.join(pd.DataFrame(svd.components_[i],columns=['comp_'+str(i)]))

print("Principales componentes:")
for x in componentes_SVD.drop(columns=['feature']).columns:
    aux=componentes_SVD.sort_values(by=[x],ascending=False)
    mask=componentes_SVD[x]>0.10
    aux=aux.loc[mask,:]
    print(x+":",','.join(aux['feature'][0:15]))
    print("")

Principales componentes:
comp_0: bueno,mas,si,hace,tambien,puede,paso,ahora,caso,hablando,dice,quiero,argentina,estan,cuarentena

comp_1: supermercado,clinica,abrir,puede,policia,imagen,nico,franco,pagar

comp_2: cobrar,cuarentena,frontera,provincia,tambien,situacion

comp_3: mas,hace,convocando,va,van



In [82]:
prueba = pd.DataFrame(Tfidf_encoding.todense(),columns=features_names)
prueba.index = file
prueba= prueba.T

In [83]:
for x in prueba.columns:
    print(x)
    print(prueba[x].sort_values(ascending=False)[0:10])
    print("")

C5N-8-11
original
mas        0.234492
hace       0.211332
bueno      0.208437
si         0.208437
paso       0.167908
habia      0.141853
tambien    0.136063
dice       0.133168
dos        0.130273
mentira    0.126840
Name: C5N-8-11, dtype: float64

C5N-8-14
original
mas           0.300684
hace          0.248927
si            0.243998
bueno         0.214423
tambien       0.194706
caso          0.157736
puede         0.138019
cuarentena    0.120767
va            0.120767
paso          0.118302
Name: C5N-8-14, dtype: float64

TN-8-11
original
bueno       0.225680
si          0.208320
puede       0.202533
mas         0.179387
tambien     0.162027
hablando    0.150453
hace        0.150453
paso        0.144667
quiero      0.138880
dice        0.127307
Name: TN-8-11, dtype: float64

TN-8-14
original
bueno         0.290251
tambien       0.223838
si            0.199240
mas           0.196781
puede         0.169723
ahora         0.164804
hace          0.159884
cuarentena    0.154965
argentina  

original,aa,aaa,aahora,abajo,abasto,abdomen,abdominales,abertis,abierto,abismo,...,ymuy,you,youtube,ypor,yreino,yumbo,zaragoza,zelanda,zona,zonda
C5N-8-11,0.002895,0.000000,0.000000,0.017495,0.000000,0.000000,0.000000,0.000000,0.008685,0.011095,...,0.005548,0.000000,0.000000,0.000000,0.005548,0.000000,0.005548,0.000000,0.000000,0.005548
C5N-8-14,0.002465,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.004723,0.012323,0.000000,...,0.000000,0.004723,0.000000,0.004723,0.000000,0.000000,0.000000,0.000000,0.030146,0.000000
TN-8-11,0.005787,0.011089,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.005787,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.035390,0.000000
TN-8-14,0.004920,0.000000,0.004714,0.003716,0.009427,0.004714,0.009427,0.000000,0.004920,0.000000,...,0.000000,0.000000,0.004714,0.000000,0.000000,0.014141,0.000000,0.004714,0.012035,0.000000
